In [1]:
# !pip install torchsummary

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torchsummary import summary


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'{epoch} - Training loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Base Network:

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)
    
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/var/folders/nv/qt1089d14r50f78yg8tx_prc0000gp/T/ipykernel_1093/1857902093.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


# Network:1

In [5]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)
        # self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(16, 32, 3, padding=0)
        self.conv6 = nn.Conv2d(32, 32, 3, padding=0)
        self.conv7 = nn.Conv2d(32, 64, 3, padding=0)
        self.conv8 = nn.Conv2d(64, 64, 3, padding=0)
        self.conv9 = nn.Conv2d(64, 128, 3, padding=0)
        self.conv10 = nn.Conv2d(128, 128, 3, padding=0)
        self.conv11 = nn.Conv2d(128, 128, 3, padding=0)
        self.conv12 = nn.Conv2d(128, 128, 3, padding=0)
        self.conv13 = nn.Conv2d(128, 128, 3, padding=0)
        self.conv14 = nn.Conv2d(128, 128, 3, padding=0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.conv14(x)
        # x = x.view(-1,10)
        print(x.shape)
        # x = nn.Linear(x, out_features=10, bias=True)

        return F.log_softmax(x)
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        # x = x.view(-1, 10)

model = Net1().to(device)
summary(model, input_size=(1, 28, 28))

torch.Size([2, 128, 8, 8])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2            [-1, 8, 28, 28]             584
            Conv2d-3           [-1, 16, 28, 28]           1,168
            Conv2d-4           [-1, 16, 28, 28]           2,320
            Conv2d-5           [-1, 32, 26, 26]           4,640
            Conv2d-6           [-1, 32, 24, 24]           9,248
            Conv2d-7           [-1, 64, 22, 22]          18,496
            Conv2d-8           [-1, 64, 20, 20]          36,928
            Conv2d-9          [-1, 128, 18, 18]          73,856
           Conv2d-10          [-1, 128, 16, 16]         147,584
           Conv2d-11          [-1, 128, 14, 14]         147,584
           Conv2d-12          [-1, 128, 12, 12]         147,584
           Conv2d-13          [-1, 128, 10, 10]         147,584
           C

/var/folders/nv/qt1089d14r50f78yg8tx_prc0000gp/T/ipykernel_1093/2877682149.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


# Network-2:

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)  # Input: 28x28, Output: 28x28
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)  # Input: 28x28, Output: 28x28
        self.pool1 = nn.MaxPool2d(2, 2)  # Input: 28x28, Output: 14x14
        
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)  # Input: 14x14, Output: 14x14
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)  # Input: 14x14, Output: 14x14
        self.pool2 = nn.MaxPool2d(2, 2)  # Input: 14x14, Output: 7x7
        
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)  # Input: 7x7, Output: 7x7
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)  # Input: 7x7, Output: 7x7
        
        self.fc1 = nn.Linear(128 * 7 * 7, 256)  # Flatten from 128x7x7 to 256 neurons
        self.fc2 = nn.Linear(256, 10)  # Output layer for 10 classes

    def forward(self, x):
        # Apply convolutions with ReLU and pooling
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        print(x.size())
        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 128*7*7)
        
        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation here; handled by softmax below
        
        return F.log_softmax(x, dim=1)  # Log softmax over class dimension
    
model = Net2().to(device)
summary(model, input_size=(1, 28, 28))

torch.Size([2, 128, 7, 7])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2           [-1, 16, 28, 28]           1,168
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           4,640
            Conv2d-5           [-1, 64, 14, 14]          18,496
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7            [-1, 128, 7, 7]          73,856
            Conv2d-8            [-1, 128, 7, 7]         147,584
            Linear-9                  [-1, 256]       1,605,888
           Linear-10                   [-1, 10]           2,570
Total params: 1,854,282
Trainable params: 1,854,282
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.43
Par

- with SGD

In [32]:

model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

loss=0.1337766796350479 batch_id=468: 100%|██████████| 469/469 [01:18<00:00,  5.98it/s]  



Test set: Average loss: 0.0955, Accuracy: 9681/10000 (97%)



loss=0.07053033262491226 batch_id=468: 100%|██████████| 469/469 [01:19<00:00,  5.90it/s] 



Test set: Average loss: 0.0598, Accuracy: 9800/10000 (98%)



...


Test set: Average loss: 0.0357, Accuracy: 9883/10000 (99%)



loss=0.002598205115646124 batch_id=468: 100%|██████████| 469/469 [01:18<00:00,  6.01it/s]  



Test set: Average loss: 0.0366, Accuracy: 9901/10000 (99%)



loss=0.0027463010046631098 batch_id=468: 100%|██████████| 469/469 [01:19<00:00,  5.93it/s] 



Test set: Average loss: 0.0379, Accuracy: 9896/10000 (99%)



- with ADAM

In [9]:

model = Net2().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Training loss=0.24829816818237305 batch_id=468: 100%|██████████| 469/469 [01:20<00:00,  5.82it/s]



Test set: Average loss: 0.1499, Accuracy: 9512/10000 (95%)



Training loss=0.061519164592027664 batch_id=468: 100%|██████████| 469/469 [01:21<00:00,  5.77it/s]



Test set: Average loss: 0.0871, Accuracy: 9711/10000 (97%)



Training loss=0.04699212685227394 batch_id=468: 100%|██████████| 469/469 [01:20<00:00,  5.82it/s] 



Test set: Average loss: 0.0612, Accuracy: 9809/10000 (98%)



Training loss=0.006802314426749945 batch_id=468: 100%|██████████| 469/469 [01:19<00:00,  5.87it/s]



Test set: Average loss: 0.0576, Accuracy: 9809/10000 (98%)



Training loss=0.0237752553075552 batch_id=468: 100%|██████████| 469/469 [01:21<00:00,  5.75it/s]  



Test set: Average loss: 0.0444, Accuracy: 9848/10000 (98%)



# Network -3 :

In [ ]:
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)  # Input: 28x28, Output: 28x28
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)  # Input: 28x28, Output: 28x28
        self.pool1 = nn.MaxPool2d(2, 2)  # Input: 28x28, Output: 14x14
        
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)  # Input: 14x14, Output: 14x14
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)  # Input: 14x14, Output: 14x14
        self.pool2 = nn.MaxPool2d(2, 2)  # Input: 14x14, Output: 7x7
        
        self.conv5 = nn.Conv2d(16, 32, 3, padding=1)  # Input: 7x7, Output: 7x7
        self.conv6 = nn.Conv2d(32, 32, 3, padding=1)  # Input: 7x7, Output: 7x7
        
        self.fc1 = nn.Linear(32 * 7 * 7, 256)  # Flatten from 128x7x7 to 256 neurons
        self.fc2 = nn.Linear(256, 10)  # Output layer for 10 classes

    def forward(self, x):
        # Apply convolutions with ReLU and pooling
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        # print(x.size())
        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 128*7*7)
        
        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation here; handled by softmax below
        
        return F.log_softmax(x, dim=1)  # Log softmax over class dimension
    
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net3().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2            [-1, 8, 28, 28]             584
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
            Conv2d-5           [-1, 16, 14, 14]           2,320
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 32, 7, 7]           4,640
            Conv2d-8             [-1, 32, 7, 7]           9,248
            Linear-9                  [-1, 256]         401,664
           Linear-10                   [-1, 10]           2,570
Total params: 422,274
Trainable params: 422,274
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.19
Params size (MB): 1.61
Estimated T

In [ ]:

model = Net3().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

1 - Training loss=0.23439884185791016 batch_id=468: 100%|██████████| 469/469 [00:43<00:00, 10.74it/s]



Test set: Average loss: 0.2475, Accuracy: 9238/10000 (92%)



2 - Training loss=0.18272435665130615 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.50it/s]



Test set: Average loss: 0.1359, Accuracy: 9556/10000 (96%)



3 - Training loss=0.3022434413433075 batch_id=468: 100%|██████████| 469/469 [00:44<00:00, 10.51it/s]  



Test set: Average loss: 0.1000, Accuracy: 9659/10000 (97%)



4 - Training loss=0.024944975972175598 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.54it/s]



Test set: Average loss: 0.0719, Accuracy: 9762/10000 (98%)



5 - Training loss=0.08905235677957535 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.50it/s] 



Test set: Average loss: 0.0627, Accuracy: 9798/10000 (98%)



# Network -4 :

In [5]:
class Net4(nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)  # Input: 28x28, Output: 28x28
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)  # Input: 28x28, Output: 28x28
        self.pool1 = nn.MaxPool2d(2, 2)  # Input: 28x28, Output: 14x14
        
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)  # Input: 14x14, Output: 14x14
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)  # Input: 14x14, Output: 14x14
        self.pool2 = nn.MaxPool2d(2, 2)  # Input: 14x14, Output: 7x7
        
        self.conv5 = nn.Conv2d(16, 32, 3, padding=1)  # Input: 7x7, Output: 7x7
        self.conv6 = nn.Conv2d(32, 32, 3, padding=1)  # Input: 7x7, Output: 7x7
        
        self.fc1 = nn.Linear(32 * 7 * 7, 50)  # Flatten from 128x7x7 to 256 neurons
        self.fc2 = nn.Linear(50, 10)  # Output layer for 10 classes

    def forward(self, x):
        # Apply convolutions with ReLU and pooling
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        # print(x.size())
        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 128*7*7)
        
        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation here; handled by softmax below
        
        return F.log_softmax(x, dim=1)  # Log softmax over class dimension
    
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net4().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2            [-1, 8, 28, 28]             584
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
            Conv2d-5           [-1, 16, 14, 14]           2,320
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 32, 7, 7]           4,640
            Conv2d-8             [-1, 32, 7, 7]           9,248
            Linear-9                   [-1, 50]          78,450
           Linear-10                   [-1, 10]             510
Total params: 97,000
Trainable params: 97,000
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.19
Params size (MB): 0.37
Estimated Tot

In [14]:

model = Net4().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

1 - Training loss=0.48026803135871887 batch_id=468: 100%|██████████| 469/469 [00:43<00:00, 10.90it/s]



Test set: Average loss: 0.3544, Accuracy: 8996/10000 (90%)



2 - Training loss=0.23670661449432373 batch_id=468: 100%|██████████| 469/469 [00:42<00:00, 11.14it/s]



Test set: Average loss: 0.1950, Accuracy: 9411/10000 (94%)



3 - Training loss=0.22457338869571686 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.62it/s]



Test set: Average loss: 0.1391, Accuracy: 9573/10000 (96%)



4 - Training loss=0.1355208456516266 batch_id=468: 100%|██████████| 469/469 [00:41<00:00, 11.23it/s]  



Test set: Average loss: 0.1155, Accuracy: 9647/10000 (96%)



5 - Training loss=0.02095029503107071 batch_id=468: 100%|██████████| 469/469 [00:42<00:00, 11.04it/s] 



Test set: Average loss: 0.0937, Accuracy: 9695/10000 (97%)



- Lets increase the number of epochs and observe

In [15]:

model = Net4().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

1 - Training loss=0.31195130944252014 batch_id=468: 100%|██████████| 469/469 [00:43<00:00, 10.68it/s]



Test set: Average loss: 0.3531, Accuracy: 8958/10000 (90%)



2 - Training loss=0.26081863045692444 batch_id=468: 100%|██████████| 469/469 [00:43<00:00, 10.76it/s]



Test set: Average loss: 0.2284, Accuracy: 9274/10000 (93%)



3 - Training loss=0.1638379544019699 batch_id=468: 100%|██████████| 469/469 [00:43<00:00, 10.69it/s] 


...


Test set: Average loss: 0.0553, Accuracy: 9825/10000 (98%)



19 - Training loss=0.0316123440861702 batch_id=468: 100%|██████████| 469/469 [00:44<00:00, 10.62it/s]   



Test set: Average loss: 0.0477, Accuracy: 9857/10000 (99%)



20 - Training loss=0.01684548333287239 batch_id=468: 100%|██████████| 469/469 [00:47<00:00,  9.96it/s]  



Test set: Average loss: 0.0495, Accuracy: 9841/10000 (98%)



- Increase learning rate from `1e-4 to 1e-2`

In [6]:

model = Net4().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

1 - Training loss=0.1377662569284439 batch_id=468: 100%|██████████| 469/469 [00:42<00:00, 11.12it/s]  



Test set: Average loss: 0.0889, Accuracy: 9729/10000 (97%)



2 - Training loss=0.10433440655469894 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.67it/s] 



Test set: Average loss: 0.0710, Accuracy: 9779/10000 (98%)



...


Test set: Average loss: 0.0761, Accuracy: 9800/10000 (98%)



9 - Training loss=0.049256596714258194 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.51it/s] 



Test set: Average loss: 0.0641, Accuracy: 9812/10000 (98%)



10 - Training loss=0.10781421512365341 batch_id=468: 100%|██████████| 469/469 [00:41<00:00, 11.35it/s] 



Test set: Average loss: 0.0665, Accuracy: 9803/10000 (98%)



# Network - 5:

In [7]:
class Net5(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net5, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3) # 26
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3) # 24
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3) # 22
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3) # 20
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x), 2)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(self.conv3(x), 2)
        x = F.relu(F.max_pool2d(self.conv4(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net5().to(device)
summary(model, input_size=(1, 28, 28))

RuntimeError: shape '[-1, 320]' is invalid for input of size 8192